**Paraphrase Identification** : Paraphrase identification is a natural language processing (NLP) problem that involves the determination of whether two text segments have the same meaning. 

**Dataset Used** : [Paraphrase Adversaries from Word Scrambling](https://github.com/google-research-datasets/paws)

This dataset contains 108,463 human-labeled and 656k noisily labeled pairs that feature the importance of modeling structure, context, and word order information for the problem of paraphrase identification. The dataset has two subsets, one based on Wikipedia and the other one based on the Quora Question Pairs (QQP) dataset.



In [1]:
!pip install datasets 

     |████████████████████████████████| 163kB 7.5MB/s 
     |████████████████████████████████| 17.7MB 210kB/s 
     |████████████████████████████████| 245kB 55.8MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [2]:
!pip install transformers

     |████████████████████████████████| 1.8MB 7.2MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
     |████████████████████████████████| 2.9MB 49.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b9b6a8491324e61e0627bbb1f86c5daee07c460bffa640ca602c66506bf1e22e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [24]:
from datasets import load_dataset, load_metric
import numpy as np

dataset = load_dataset('paws','labeled_final')

Reusing dataset paws (/root/.cache/huggingface/datasets/paws/labeled_final/1.1.0/27cded16baeb31147f86d850889b9cbdd249494ab53c1c4f524265aab4c10fc4)


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [5]:
dataset.column_names

{'test': ['id', 'label', 'sentence1', 'sentence2'],
 'train': ['id', 'label', 'sentence1', 'sentence2'],
 'validation': ['id', 'label', 'sentence1', 'sentence2']}

In [6]:
dataset.shape

{'test': (8000, 4), 'train': (49401, 4), 'validation': (8000, 4)}

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

Let's get an idea of the data. 

In [8]:
show_random_elements(dataset["train"])

,id,label,sentence1,sentence2
0,26244,0,"The 86th Airlift Squadron is part of 309th Airlift Wing in Air Base Chièvres , Belgium .","The 309th Airlift Squadron is part of the 86th Airlift Wing at Chièvres Air Base , Belgium ."
1,33160,0,Hussain received 432 votes and his only rival Wajihuddin Ahmed secured 77 .,"Hussain received 432 votes , and his only rival , Wajihuddin Ahmed , was 77 ."
2,4704,1,Both traced the continuing influence on literary and cultural studies of the kinds of cultural materialism developed by Williams and his successors .,Both highlighted the continuing influence of the forms of cultural materialism developed by Williams and his successors on literary and cultural studies .
3,47808,0,"In Finnmark , he completed several of the paintings he had outlined on his 1832 Stockholm tour .",In Stockholm he completed some of the paintings he had outlined on his Finnmark tour in 1832 .
4,3711,0,"Serkan Yalçan ( born November 2 , 1982 ) is a Turkish professional footballer who plays in the TFF First League as a defender of Akhisar Belediyespor .","Serkan Yalçan ( born November 2 , 1982 ) is a Turkish football professional who plays as a defender for the TFF First League in Akhisar Belediyespor ."
5,11898,0,"`` , Williams turned heel and reformed the British Invasion with Magnus by attacking the team of Eric Young and Orlando Jordan .",Magnus turned the heel and reformed the British invasion of Williams by attacking Eric Young and Orlando Jordan 's team .
6,40567,1,"Crocker moved from Natchez , Mississippi to Vidalia , the headquarters of the Concordia Parish , and crossed the lower Ouachita in the Black River section .","Crocker crossed from Natchez , Mississippi , to Vidalia , the seat of the Concordia Parish , and moved towards the lower Ouachita in the Black River section ."
7,35758,0,"Anti-dsdNA biological therapies , such as adalimumab , infliximab and etanercept , can often induce the production of Anti-TNFα antibodies .","Biological therapies such as Adalimumab , Infliximab and Etanercept can often induce the production of anti-DSNNA antibodies ."
8,10665,1,Ezra 's family came originally as immigrants from Iraq and settled in Palestine .,Ezra 's family originally came as immigrants from Iraq and settled down in Palestine .
9,6266,1,"It is available in areas like Ballymun , Finglas , Cabra , Phibsboro and Castleknock .","It is available in areas including Phibsboro and Castleknock , Finglas , Cabra and Ballymun ."


# Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.



In [9]:
# parameters

task = "mrpc"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [10]:
metric = load_metric('glue', task)

In [11]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
""", stored examples: 0)

## How to compute metrics of the given task

```python

import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)
```

# Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

* we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.

* That vocabulary will be cached, so it's not downloaded again the next time we run the cell.



In [12]:
'''
Here we will use the pretrained tokenizer
'''

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along use_fast=True to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [13]:
tokenizer("I need a flight from New York to Paris", "I need a flight from Paris to New York.")

{'input_ids': [101, 1045, 2342, 1037, 3462, 2013, 2047, 2259, 2000, 3000, 102, 1045, 2342, 1037, 3462, 2013, 3000, 2000, 2047, 2259, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
task_to_keys = {
    "mrpc": ("sentence1", "sentence2")
}

In [15]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .
Sentence 2: In October 1560 , he secretly met with the English ambassador , Nicolas Throckmorton , in Paris , and asked him for a passport to return to Scotland through England .


In [16]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [17]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 1999, 3000, 1010, 1999, 2255, 29185, 1010, 2002, 10082, 2777, 1996, 2394, 6059, 1010, 9473, 16215, 16901, 5302, 11715, 1010, 4851, 2032, 2005, 1037, 12293, 2000, 2709, 2000, 2563, 2083, 3885, 1012, 102, 1999, 2255, 29185, 1010, 2002, 10082, 2777, 2007, 1996, 2394, 6059, 1010, 9473, 16215, 16901, 5302, 11715, 1010, 1999, 3000, 1010, 1998, 2356, 2032, 2005, 1037, 12293, 2000, 2709, 2000, 3885, 2083, 2563, 1012, 102], [101, 1996, 6452, 2161, 1997, 3339, 1011, 1011, 6146, 2001, 1996, 13293, 2161, 1997, 1996, 2120, 3455, 2523, 1012, 102, 1996, 3339, 1011, 1011, 6146, 2161, 1997, 1996, 2120, 3455, 2523, 2001, 1996, 13293, 2161, 1997, 1996, 6452, 1012, 102], [101, 2045, 2024, 2036, 3563, 10287, 1010, 2270, 6337, 14379, 1998, 2622, 10287, 1012, 102, 2045, 2024, 2036, 2270, 10287, 1010, 6337, 3563, 10287, 1010, 1998, 2622, 10287, 1012, 102], [101, 2043, 12435, 6165, 1997, 4834, 2064, 2022, 5224, 1010, 1996, 3463, 2024, 2152, 1012, 102, 1996, 3463, 2024, 2152, 2043, 12435, 4

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [18]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [19]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

In [20]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the Trainer to load the best model it saved (according to metric_name) at the end of training.

The last thing to define for our Trainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B)

In [21]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [22]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Runtime,Samples Per Second
1,0.226000,0.327896,0.897125,0.887152,19.320900,414.059000
2,0.165600,0.351926,0.903375,0.895016,19.392400,412.533000
3,0.137300,0.424695,0.906250,0.897931,19.356700,413.294000
4,0.106100,0.493496,0.906500,0.898148,19.397700,412.420000
5,0.090000,0.493496,0.906500,0.898148,19.445800,411.400000


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=15440, training_loss=0.14966470812268826, metrics={'train_runtime': 2121.1755, 'train_samples_per_second': 7.279, 'total_flos': 7657737828983940, 'epoch': 5.0})

In [26]:
trainer.evaluate()

{'epoch': 5.0,
 'eval_accuracy': 0.9065,
 'eval_f1': 0.898148148148148,
 'eval_loss': 0.49349597096443176,
 'eval_runtime': 20.0746,
 'eval_samples_per_second': 398.514}

# Hyperparameter search

In [27]:
! pip install optuna
! pip install ray[tune]

     |████████████████████████████████| 286kB 7.2MB/s 
     |████████████████████████████████| 163kB 12.5MB/s 
     |████████████████████████████████| 81kB 6.3MB/s 
     |████████████████████████████████| 481kB 12.6MB/s 
     |████████████████████████████████| 133kB 21.0MB/s 
     |████████████████████████████████| 112kB 21.9MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=d7d79ebd59120f6b3044f6737620d68f7f3b557f5339543fce5d4bd8c8d96f0e
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=b355d9e218cd12fbb23ed62fbd7b16a411626d50b7b4d2ef3da2cafc4864f5b4
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for pyperclip: filename=pyperclip-1.8.1-cp36-none-any.whl size=11120 sha

In [28]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [30]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"].shard(index=1, num_shards=10),
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [31]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2021-01-16 04:59:33,450] A new study created in memory with name: no-name-c1064b70-10d8-47b7-bc20-9343bfa10443
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpo

Epoch,Training Loss,Validation Loss,Accuracy,F1,Runtime,Samples Per Second
1,0.689800,0.685803,0.551375,0.108323,19.104100,418.758000
2,0.675100,0.685737,0.552250,0.139769,19.286900,414.789000


[I 2021-01-16 05:02:36,069] Trial 0 finished with value: 434.7679194524496 and parameters: {'learning_rate': 6.109829200853746e-06, 'num_train_epochs': 2, 'seed': 27, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 434.7679194524496.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Runtime,Samples Per Second
1,0.695500,0.684901,0.557625,0.000000,19.245900,415.672000
2,0.692000,0.690001,0.557625,0.000000,19.317300,414.137000
3,0.672700,0.745681,0.524750,0.498814,19.432200,411.687000
4,0.605100,0.838788,0.532625,0.472414,19.484100,410.591000


[I 2021-01-16 05:07:16,928] Trial 1 finished with value: 431.080139279667 and parameters: {'learning_rate': 7.947647113349044e-05, 'num_train_epochs': 4, 'seed': 20, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 434.7679194524496.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

...

[I 2021-01-16 05:19:40,617] Trial 7 pruned. 
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_class

Epoch,Training Loss,Validation Loss,Accuracy,F1,Runtime,Samples Per Second
1,0.694500,0.688699,0.556750,0.053390,19.476200,410.758000


[I 2021-01-16 05:20:49,256] Trial 8 pruned. 
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_class

Epoch,Training Loss,Validation Loss,Accuracy,F1,Runtime,Samples Per Second
1,No log,0.687378,0.555625,0.020391,19.497700,410.305000


[I 2021-01-16 05:21:47,341] Trial 9 pruned. 


In [32]:
best_run

BestRun(run_id='0', objective=434.7679194524496, hyperparameters={'learning_rate': 6.109829200853746e-06, 'num_train_epochs': 2, 'seed': 27, 'per_device_train_batch_size': 4})